In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from os import listdir, getenv
import sqlalchemy as sa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [22]:
# Import the querying module
from flight_querying import query_flights
import pandas as pd

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [23]:
# Flight IDs to include
flight_ids = [4620, 4929, 4940, 5019, 5021, 5034]

# Fetch data for specified flight IDs
data_frames = [db_connect.connect_flight_for_ml_data_label(flight_id) for flight_id in flight_ids]

# Concatenate data frames and shuffle the data
all_data = pd.concat(data_frames, axis=0).sample(frac=1, random_state=42)

In [24]:

#multi-nomial encoding 
le = LabelEncoder()
all_data['y'] = le.fit_transform(all_data['exercise'])
print(all_data.head())

            time    id exercise  soc  cell_temperature  motor_rpm  \
13078  21.906906  5034   cruise   80                20       2040   
3824    6.452442  5019       NA   99                17          0   
9138   15.273406  4940    climb   79                26       2338   
5358    9.002953  4620       NA  100                23          0   
11410  19.117159  5034    climb   91                19       2244   

       motor_power  motor_temperature  indicated_air_speed  pressure_altitude  \
13078           21          53.933147            85.390571         890.998779   
3824             0          21.725458             0.000000         303.684998   
9138            41          65.415855            74.731521        1023.144287   
5358             0          26.487469             0.000000         224.602600   
11410           44          46.848240            72.343706         522.239807   

       ground_speed  outside_air_temperature  inverter_temperature     pitch  \
13078          88.

In [25]:
# Split the data into train and test sets
train_data, test_data = train_test_split(all_data, test_size=0.5, random_state=42)

# Set up train data
train_y = train_data["y"].to_numpy()
train_x = train_data.drop(columns=["exercise","id", "y"])

# Set up test data
test_y = test_data["y"].to_numpy()
test_x = test_data.drop(columns=["exercise", "id","y"])

In [26]:
print(f"Length train_x = {len(train_x)} \n Length train_y = {len(train_y)}")
print(f"Length test_x = {len(test_x)} \n Length test_y = {len(test_y)}")

Length train_x = 88034 
 Length train_y = 88034
Length test_x = 88035 
 Length test_y = 88035


In [27]:
train_x

,time,soc,cell_temperature,motor_rpm,motor_power,motor_temperature,indicated_air_speed,pressure_altitude,ground_speed,outside_air_temperature,inverter_temperature,pitch,roll
11328,18.941536,71,26,1170,0,55.990517,73.456477,1064.456299,71.1,20.0,35.395370,3.194920,-0.420180
29142,48.778612,35,12,0,0,28.022661,0.000000,312.330933,1.4,11.5,28.439835,2.829932,0.566098
394,0.702049,100,18,0,0,15.110507,0.000000,323.038788,0.0,16.0,19.757511,2.603294,0.496127
5719,9.546005,97,23,1173,7,31.950922,20.799833,314.835693,0.0,26.5,36.270409,4.369888,-0.269310
24572,41.143868,30,15,2212,47,45.553085,64.535853,465.452820,56.2,9.0,33.728527,10.099205,-2.519308
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16770,28.046831,55,24,2010,30,51.164127,60.560229,1040.916626,56.7,19.0,37.334377,9.252363,5.535706
27359,45.801679,24,15,911,4,36.045177,28.383542,305.340088,29.6,8.5,26.639986,1.607027,-0.711601
15210,25.483659,65,15,1192,0,38.398270,67.290296,913.715210,69.0,8.0,26.348877,0.111682,1.196560
21544,36.081695,42,13,1212,4,30.368723,55.789655,334.411774,56.4,8.5,24.150352,-0.575589,0.760856


In [28]:
train_y

array([5, 0, 0, ..., 3, 4, 3])

In [29]:
test_x

,time,soc,cell_temperature,motor_rpm,motor_power,motor_temperature,indicated_air_speed,pressure_altitude,ground_speed,outside_air_temperature,inverter_temperature,pitch,roll
26643,44.594837,42,17,2269,47,50.245621,71.349741,558.984497,62.5,13.0,36.259029,9.137207,-11.866501
6693,11.252713,86,18,1944,15,48.322350,93.548783,797.952881,90.4,13.5,34.349075,-4.697657,-1.730736
14120,23.613119,63,25,1790,19,52.359505,69.206841,1003.988281,61.2,20.0,36.628326,4.682217,-9.258916
13019,21.828115,64,16,1968,14,43.627144,89.614788,974.033386,71.6,6.0,29.081947,-3.466079,0.295602
15663,26.264315,63,18,1935,19,43.925282,83.062707,770.048950,85.5,14.0,32.955479,-0.364615,0.711140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11477,19.249717,68,16,1273,0,46.733337,72.397562,1057.953369,96.1,6.0,29.068392,-1.226546,1.611414
7603,12.776225,83,18,1881,20,45.908138,72.049480,782.161255,74.7,13.5,33.726181,1.692268,-0.576937
1237,2.126370,99,15,0,0,12.843011,0.000000,281.591583,0.0,13.0,16.394247,1.857302,-0.208379
23866,39.963802,33,15,1230,5,37.421116,55.967126,343.133942,56.3,9.0,27.995983,-0.523597,-6.504545


In [30]:
test_y

array([1, 2, 2, ..., 0, 4, 1])

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [32]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(train_x, train_y)

/Users/nayeema/envs/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [33]:
# Make predictions on the test set
predictions = model.predict(test_x)

# Evaluate the model
accuracy = accuracy_score(test_y, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display additional classification metrics
print("Classification Report:")
print(classification_report(test_y, predictions))

Accuracy: 0.76
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89     31767
           1       0.81      0.74      0.78      8766
           2       0.66      0.90      0.76     21215
           3       0.56      0.34      0.43      6898
           4       0.72      0.72      0.72      9104
           5       0.60      0.46      0.52       896
           6       0.00      0.00      0.00       308
           7       0.00      0.00      0.00      2206
           8       0.46      0.21      0.29      3220
           9       0.86      0.89      0.88      3655

    accuracy                           0.76     88035
   macro avg       0.56      0.52      0.53     88035
weighted avg       0.74      0.76      0.74     88035



## Saving & Loading The Model

In [34]:
import joblib

# Assuming you've trained and named your Logistic Regression model 'logreg_model'

# Save the model to a file
model_filename = 'multinomial_logreg_model.joblib'
joblib.dump(model, model_filename)

print(f"Model saved to {model_filename}")


Model saved to multinomial_logreg_model.joblib


In [35]:
loaded_model = joblib.load(model_filename)